In [1]:
# add parent dir to syspath
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
root_dir = os.path.dirname(parent_dir)
sys.path.insert(0, root_dir)

In [2]:
from imageGenerators import load_realdata, distortions
from imageGenerators.imgGen_simple import synth_generator
from imageGenerators.imgGen_augmented import synth_generator as synth_generator_new

from models.cyclegan_modified.cyclegan import cyclegan
from imageGenerators import load_realdata

import tensorflow as tf
import cv2
from pathlib import Path
from matplotlib import pyplot as plt
import numpy as np
from time import time
import re
import random
import pickle

import lpips_interface
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [3]:
# assert model is running on gpu
from tensorflow.python.client import device_lib
devlist = str(device_lib.list_local_devices())
assert "GPU:0" in devlist

In [4]:
# Path to i2i-model to evaluate
i2i_model = Path("D:/m2/savedmodels/goals/q1_n_images/n128_epochs16")
i2i_epoch = 11
n_digits = 5


n_inputs = 100
n_pairsPerInput = 19 # angelehnt an gewisse Paper. Warum genau 19?

# zum zeit sparen: 50,10 => 500 Bilder ~ 1.2 Minuten

n_pairs = n_inputs * n_pairsPerInput

# split set of pairs into multiple parts and calculate lpips for each split
splits = 10
assert n_pairs % splits == 0

spielt bildgröße eine Rolle bei Berechnung?

In [5]:
# dimensions of input to wmn-model
fcsrn_height = 48; fcsrn_width = 160; fcsrn_channels = 1
fcsrn_shape = (fcsrn_height, fcsrn_width, fcsrn_channels)
fcsrn_dims = (fcsrn_width, fcsrn_height)
# get shape of inputs for i2i-model
inputshapePath = i2i_model / "inputshape"
i2i_shape = [int(s) for s in inputshapePath.read_text().split(",")]
i2i_dims = (i2i_shape[1], i2i_shape[0])

>100 input images and sample 19 output pairs per input, which amounts to 1900 pairs in total

mehrdeutig:
+ a: generiere je input 38 Bilder. Diese 3800 Bilder mische und bilde 1900 Paare.
+ b: generiere je input 38 Bilder. Bilde daraus 19 Paare. Das für jedes Input -> 1900 Paare

dh. bestehen Paare aus Bildern des selben Inputs, oder aus zufälligen Inputs?

# a: paare gemischt

In [6]:
def showImage(image, normalized=False):
    if len(image.shape) == 3 and image.shape[2] == 1:
        image = image[:,:,0]
    #image = image.astype(int)
    if normalized:
        plt.imshow(image, cmap="gray", vmin=-1, vmax=1)
    else:
        plt.imshow(image, cmap="gray", vmin=0, vmax=255)
    plt.show()
def showImages(images, n, labels=None, normalized=False):
    for i in range(min(n, len(images))):
        if not labels is None:
            print(labels[i])
        image = images[i]
        showImage(image, normalized)

# prepares images for cyclegan
# input: shape [n,h,w], dtype int[0,255]
# output: shape [n,h,w,c], dtype float32[-1,1]
def prepareImages(images, resultchannels=1):
    # norm to -1,1
    images = images.astype("float32")
    images = (images / 127.5) - 1
    # add channel-dims
    shape = images.shape
    if len(shape) < 4:
        shape = [d for d in shape]
        shape.append(1)
        images = np.reshape(images, shape)
    
    if resultchannels==3:
        images = tf.tile(image, [1, 1, 1, 3])
        
    return images

In [ ]:
def lpips_cgan(from_labels=True):
    synthGenerator = synth_generator_new()
    cgModel = cyclegan(i2i_shape,-1,-1, "mse", checkpoint_path=i2i_model, load_checkpoint_after_epoch=i2i_epoch)
    
    if not from_labels:
        labels_synth = np.random.randint(0,20, (n_inputs, n_digits) )
        images_synth = synthGenerator.makeImages(labels_synth, resizeTo=i2i_dims, color=True, rotate=True)
        images_synth = prepareImages(images_synth)
        d = lpips_interface.mean_distance_from_i2imodel(cgModel, images_synth, n_inputs, n_pairsPerInput, splits)
    else:        
        d = lpips_interface.mean_distance_from_i2imodel(cgModel, None, n_inputs, n_pairsPerInput, splits, start_from_labels=True, synthGen=synthGenerator, synthDims=i2i_dims, fix_offsets=True)
                                         

In [39]:
# lpips expects images normalied to [-1,1] with shape [n,h,w,3]
def prepare_for_lpips(image):
    # norm to -1,1
    image = image.astype("float32")
    image = (image / 127.5) - 1
    # add channel-dims
    shape = image.shape
    shape = [d for d in shape]
    shape.append(1)
    if len(shape) == 3:
        # add batch-dimensions if single image
        shape.insert(0, 1)
    image = np.reshape(image, shape)
    # to 3-channel
    image = tf.tile(image, [1, 1, 1, 3])    
    return image

def lpips_imageset(images, n_pairs, verbose=False):
    random_indices = np.random.randint(0,len(images), (2,n_pairs))
    images_a = images[random_indices[0]]
    images_b = images[random_indices[1]]
    
    images_a = prepare_for_lpips(images_a)
    images_b = prepare_for_lpips(images_b)
    
    stime = time()
    d = lpips_interface.distance(images_a,images_b)
    if verbose:
        print("%.2f seconds" % (time() - stime) )
    
    mean = tf.math.reduce_mean(d)
    maximum = tf.math.reduce_max(d)
    minimum = tf.math.reduce_min(d)
    if verbose:
        print("mean:\t%f" % (mean) )
        print("maximum:\t%f" % (maximum) )
        print("minimum:\t%f" % (minimum) )
    return mean

def lpips_repeatedly(images, n_pairs, n_reps):
    lpip_means = []
    for _ in range(n_reps):
        mean = lpips_imageset(images, n_pairs, verbose=False)
        lpip_means.append(mean.numpy())
    txt = "".join(["%f\t" % (mean) for mean in lpip_means ])
    print(txt)
    return lpip_means

lpips for real images

In [50]:
# how often to calculate lpips for each imageset
n_repetitions = 10
n_pairs = 3800

In [51]:
# real easy images, keepratio=True
stime = time()
images, _ = load_realdata.load_wmr_easy(n_toLoad = None, resizeTo=fcsrn_dims, keepRatio=True, processImage=None)
d_easy_keepRatio = lpips_repeatedly(images, n_pairs, n_repetitions)
print("took %.2f seconds" % (time() - stime) )

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[3800,64,11,39] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node import/Pow_1 (defined at C:\Users\andre\jupyter_ws\ganMetrics\lpips\lpips_tf.py:87) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[import/Squeeze_7/_20]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[3800,64,11,39] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node import/Pow_1 (defined at C:\Users\andre\jupyter_ws\ganMetrics\lpips\lpips_tf.py:87) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_alloc

In [ ]:
# real easy images, keepratio=False
stime = time()
images, _ = load_realdata.load_wmr_easy(n_toLoad = None, resizeTo=fcsrn_dims, keepRatio=False, processImage=None)
d_easy_dontKeepRatio = lpips_repeatedly(images, n_pairs, n_repetitions)
print("took %.2f seconds" % (time() - stime) )

In [ ]:
# real diff train, keepratio=True
stime = time()
images, _ = load_realdata.load_wmr_diff_train(n_toLoad = None, resizeTo=fcsrn_dims, keepRatio=True, processImage=None)
d_easy_keepRatio = lpips_repeatedly(images, n_pairs, n_repetitions)
print("took %.2f seconds" % (time() - stime) )

In [ ]:
# real diff train, keepratio=False
stime = time()
images, _ = load_realdata.load_wmr_diff_train(n_toLoad = None, resizeTo=fcsrn_dims, keepRatio=False, processImage=None)
d_easy_keepRatio = lpips_repeatedly(images, n_pairs, n_repetitions)
print("took %.2f seconds" % (time() - stime) )

In [ ]:
# real diff test, keepratio=True
stime = time()
images, _ = load_realdata.load_wmr_diff_test(n_toLoad = None, resizeTo=fcsrn_dims, keepRatio=True, processImage=None)
d_easy_keepRatio = lpips_repeatedly(images, n_pairs, n_repetitions)
print("took %.2f seconds" % (time() - stime) )

In [ ]:
# real diff test, keepratio=False
stime = time()
images, _ = load_realdata.load_wmr_diff_test(n_toLoad = None, resizeTo=fcsrn_dims, keepRatio=False, processImage=None)
d_easy_keepRatio = lpips_repeatedly(images, n_pairs, n_repetitions)
print("took %.2f seconds" % (time() - stime) )